In [5]:
import csv
from csv import DictReader
import requests
import json


Relation types are listed at https://support.datacite.org/docs/eventdata-guide#relation-type-id. This script looks for the resources (subjects) that are related somehow to each dataset PID (object) in the given CSV or TXT file.

I've ignoring the following relation types since I'm fairly confident that these aren't used to describe the relationship between two different studies:
- is-version-of
- is-new-version-of
- is-previous-version-of
- is-part-of
- has-part
- is-identical-to

In [23]:
relationships = [
    'Is-cited-by',
    'cites',
    'is-supplement-to',
    'is-supplemented-by',
    'is-continued-by',
    'continues',
    'describes',
    'is-described-by',
    'has-metadata',
    'is-metadata-for',
    'has-version',
    'is-referenced-by',
    'references',
    'is-documented-by',
    'documents',
    'is-compiled-by',
    'compiles',
    'is-variant-form-of',
    'is-original-form-of',
    'is-reviewed-by',
    'reviews',
    'is-derived-from',
    'is-source-of',
    'is-required-by',
    'requires'
]


In [7]:
# datasetPIDs = [
#     'doi:10.7910/dvn/28075',
#     'doi:10.7910/dvn/ohhukh',
#     'doi:10.7910/dvn/gdf6z0']

datasetPIDs = [
    'doi:10.7910/DVN/28075']



In [36]:
# # Enter path to CSV or text file containing list of dataset PIDs
# datasetPIDFile = 'dataset_pids_QDR Main Collection_2021.08.05_18.55.01.csv'

# # Read in list of dataset PIDs from given CSV or text file
# datasetPIDs = []

# if '.csv' in datasetPIDFile:
#     with open(datasetPIDFile, mode='r', encoding='utf-8') as f:
#         csvDictReader = DictReader(f, delimiter=',')
#         for row in csvDictReader:
#             datasetPIDs.append(row['persistent_id'].rstrip())
# elif 'txt' in datasetPIDFile:
#     for datasetPID in datasetPIDFile:
#         datasetPIDs.append(datasetPID.rstrip())

In [2]:
print(len(datasetPIDs))

1


In [51]:
# Create CSV file for writing data requested from DataCite API
with open('data.csv', mode='w') as opencsvfile:
    opencsvfile = csv.writer(opencsvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    opencsvfile.writerow(['pid', 'relation-type', 'count'])


In [13]:
# Create variables for keeping count of script's progress
pidCount = 1
pidTotal = len(datasetPIDs)

for pid in datasetPIDs:
    print(pid)
    pid = pid.replace(':', '=')
    dataciteEventsAPI = 'https://api.datacite.org/events?%s&page[size]=0' % (pid)
    response = requests.get(dataciteEventsAPI)
    relationData = response.json()
    for type in relationData['meta']['relation-types']:
        print('\t%s: %s' % (type['id'], type['count']))


doi:10.7910/DVN/28075
	is-part-of: 85
	has-part: 84
	references: 4
	cites: 1
	total-resolutions-machine: 1
	total-resolutions-regular: 1
	unique-resolutions-machine: 1
	unique-resolutions-regular: 1


In [21]:
    if len(relationData['data']) > 0:
        totalPages = relationData['meta']['total-pages']

        # Initialization for paginating through Search API results and showing progress
#         pageNumber = 1
#         pageSize = 10
        condition = True
        relationCount = 0

        while condition:
#             params = params
            response = requests.get(dataciteEventsAPI)
            relationData = response.json()
            for i in relationData['data']:
                if i['attributes']['relation-type-id'] in relationships:
                    subject = i['attributes']['subj-id']
                    relationTypeId = i['attributes']['relation-type-id']
                    object = i['attributes']['obj-id']
                    occurredAt = i['attributes']['occurred-at']
                    timeStamp = i['attributes']['timestamp']
                    
                    print(subject, relationTypeId, object, occurredAt, timeStamp)
                    
                    # Write values of the three variables to a new row in the CSV
#                     with open('data.csv', mode='a') as datasets:
#                         datasets = csv.writer(datasets, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                         datasets.writerow([subject, relationTypeId, object, occurredAt, timeStamp])
                    
                    relationCount += 1

            params['page[number]'] += 1
            condition = params['page[number]'] <= totalPages
    else:
        print('No relation data found')
    pidCount += 1

Searching for metrics for https://doi.org/10.7910/dvn/28075 (1 of 3)


KeyboardInterrupt: 